In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import sqlalchemy


In [2]:
url = "https://en.wikipedia.org/wiki/List_of_European_cities_by_population_within_city_limits"
response = requests.get(url)
soup = BeautifulSoup(response.content)
rows = soup.select("table.wikitable tr")


In [3]:
soup

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of European cities by population within city limits - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"849ce791-e52b-4331-8f0e-e8fa72954500","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_European_cities_by_population_within_city_limits","wgTitle":"List of European cities by population within city limits","wgCurRevisionId":1133623951,"wgRevisionId":1133623951,"wgArticleId":35849688,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 uses Russian-langua

In [4]:
cities_dict = {
    'city':[],
    'country':[],
    'population':[],
    'lat':[],
    'long':[]
}

for i, row in enumerate(rows):
    if(i == 0 ):
        continue
    
    cities_dict['city'].append(row.select('td')[1].select('a')[0].get_text().strip())    
    cities_dict['country'].append(row.select('td')[2].select('a')[0].get_text().strip())
    cities_dict['population'].append(row.select('td')[3].select('span')[0].get_text())
    cities_dict['lat'].append(row.select('td')[7].select('span.geo-dec')[0].get_text().split()[0])
    cities_dict['long'].append(row.select('td')[7].select('span.geo-dec')[0].get_text().split()[1])

In [5]:
city_df = pd.DataFrame(cities_dict)

In [6]:
city_df[['lat_','temp']] = city_df['lat'].str.split('°',expand=True)

city_df['lat_'] = pd.to_numeric(city_df['lat_'], errors='coerce')

city_df['lat'] = city_df.apply(lambda x: x['lat_'] * (-1) if x['temp'] == 'S' else x['lat_'], axis=1)

city_df[['long_','temp2']] = city_df['long'].str.split('°',expand=True)

city_df['long_'] = pd.to_numeric(city_df['long_'], errors='coerce')

city_df['long'] = city_df.apply(lambda x: x['long_'] * (-1) if x['temp2'] == 'W' else x['long_'], axis=1)

city_df.drop(['lat_','temp','long_','temp2'],axis=1,inplace=True)

In [7]:
city_df['population'] = pd.to_numeric(city_df['population'].str.replace(',',''))

In [8]:
host = 'wbs-project3-db.c5v1hsxt2ic8.eu-central-1.rds.amazonaws.com'
schema = 'gans'
user = 'admin'
password = 'Awspark2022'
port = 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [9]:
# host = '127.0.0.1'
# schema = 'gans'
# user = 'root'
# password = 'Mysqlpark2022!'
# port = 3306
# con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'


In [10]:
city_df.to_sql('city',con=con,if_exists='append',index=False)

36